# Ranking

ДЗ№2. LambdaRank или LambdaMART.
1. Нужно самостоятельно реализовать метрики ранжирования MAP, MRR, NDCG (вместо семинара).
2. Нужно реализовать алгоритм ранжирования LambdaRank или LambdaMART. Реализовать и то и другой будет плюсом. (см. ссылки ниже).
3. Протестировать алгоритм на «стандартных» датасэтах MQ2007 и MQ2008 https://www.microsoft.com/en-us/research/project/letor-learning-rank-information-retrieval/letor-4-0/ (вычислите метрики из пункта 1)
4. Применить алгоритм к дастасэту movielens https://grouplens.org/datasets/movielens/ . Считая, что оценка «5» – это релевантный фильм, «4» – это тоже релевантный фильм (но очевидно релевантность таких фильмов ниже, чем с оценкой «5»), остальные оценки не являются релевантными. Подумайте как адаптировать формат из "стандартных" датасэтов для ранжирования к задаче рекомендаций. Не поленитесь описать это комментарием в коде.
5. Если вы делали ДЗ №1 то сравнить, ранжирование из пункта 4 с результатами «своего» SVD, если не делали то можно сравнить результаты с «чужого» SVD (например из библиотеки https://implicit.readthedocs.io/en/latest/quickstart.html , https://surprise.readthedocs.io/en/stable/getting_started.html)
 Насколько «качество» (определите качество самостоятельно - обоснуйте выбор) отличается на топ-3, топ-5, топ-10 фильмов?

Ссылки:
1. Основная статья: https://proceedings.neurips.cc/paper/2006/file/af44c4c56f385c43f2529f9b1b018f6a-Paper.pdf (запасная ссылка: https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/lambdarank.pdf )
2. Также полезно почитать:
https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/MSR-TR-2010-82.pdf 
https://www.cs.cmu.edu/~pinard/Papers/sigirfp092-donmez.pdf 
3. Примеры с формулами (но без подробностей):
https://everdark.github.io/k9/notebooks/ml/learning_to_rank/learning_to_rank.html 
4. Можно повдохновляться кодом на «плюсах»:
https://github.com/microsoft/LightGBM/blob/e79716e0b69b27c8e6e2c39f59d0db7c63242f9a/src/objective/rank_objective.hpp

## MQ2007

### RankNet

In [11]:
! python3 train.py --n_epochs 100 --name ranknet-mq2007 --model ranknet --dataset mq2007

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────
         val_map            0.42127860696226627
         val_mrr            0.4961145520210266
        val_ndcg            0.6004802179719987
────────────────────────────────────────────────────────────────────────────────
Started at 07:11:00 07-11-2023
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params
----------------------------------
0 | model | RankNet | 7.5 K 
----------------------------------
7.5 K     Trainable params
0         Non-trainable params
7.5 K     Total params
0.030     Total estimated model params size (MB)
/home/ilya/repos/recsys/.venv/lib/python3.10/

### LambdaRank

In [53]:
! python3 train.py --n_epochs 100 --name lambda-rank-mq2007 --model lambda-rank --dataset mq2007

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: ./logs/tb/lambda-rank
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────
         val_map            0.37902748197260233
         val_mrr            0.4643586277961731
        val_ndcg            0.5757187901830425
────────────────────────────────────────────────────────────────────────────────
Started at 02:37:46 07-11-2023
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params
-------------------------------------
0 | model | LambdaRank | 481   
-------------------------------------
481       Trainable params
0         Non-trainable params
481       Total params
0.002     Total estimated model param

## MQ2008

### RankNet

In [54]:
! python3 train.py --n_epochs 100 --model ranknet --name ranknet-mq2008 --dataset mq2008

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: ./logs/tb/ranknet-mq2008
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────
         val_map            0.4597659326486774
         val_mrr            0.5387672185897827
        val_ndcg            0.6045726895998438
────────────────────────────────────────────────────────────────────────────────
Started at 02:56:05 07-11-2023
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params
----------------------------------
0 | model | RankNet | 481   
----------------------------------
481       Trainable params
0         Non-trainable params
481       Total params
0.002     Total estimated model params size (MB

### LambdaRank

In [56]:
! python3 train.py --n_epochs 100 --model lambda-rank --name lambda-rank-mq2008 --dataset mq2008

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────
         val_map            0.4597659326486774
         val_mrr            0.5387672185897827
        val_ndcg            0.6045726895998438
────────────────────────────────────────────────────────────────────────────────
Started at 03:08:40 07-11-2023
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params
-------------------------------------
0 | model | LambdaRank | 481   
-------------------------------------
481       Trainable params
0         Non-trainable params
481       Total params
0.002     Total estimated model params size (MB)
/home/ilya/repos/recsys/.venv/lib/

## Movielens

### RankNet

In [12]:
! python3 train.py --n_epochs 100 --model ranknet --name ranknet-movielens --dataset movielens --user_based True --extra_metrics True

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: ./logs/tb/ranknet-movielens
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────
         val_map            0.6568911929808794
        val_map@3           0.6904889364739476
        val_map@5           0.6849393290506794
         val_mrr            0.7249204516410828
        val_ndcg            0.7013579896975339
       val_ndcg@3           0.7169095369436539
       val_ndcg@5           0.7192593655426587
────────────────────────────────────────────────────────────────────────────────
Started at 07:13:13 07-11-2023
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params
----------------------------------
0

### LambdaRank

In [13]:
! python3 train.py --n_epochs 100 --model lambda-rank --name lambda-rank-movielens --dataset movielens --user_based True --extra_metrics True

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: ./logs/tb/lambda-rank-movielens
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────
         val_map            0.6568911929808794
        val_map@3           0.6904889364739476
        val_map@5           0.6849393290506794
         val_mrr            0.7249204516410828
        val_ndcg            0.7013579896975339
       val_ndcg@3           0.7169095369436539
       val_ndcg@5           0.7192593655426587
────────────────────────────────────────────────────────────────────────────────
Started at 07:13:31 07-11-2023
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params
-----------------------------

### SVD

In [1]:
import pandas as pd
import os


def load(fname, path='/home/ilya/repos/recsys/data/ml-100k'):
    path = os.path.join(path, fname)
    return pd.read_csv(path, sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp']).drop(columns=['timestamp'])

def load_train():
    return load('ua.base')
def load_test():
    return load('ua.test')

df_train = load_train()
df_test = load_test()

In [2]:
from ranking import svd

hparams = dict(
    n_factors=100,
    n_epochs=10,
    batch_size=128,
    init_mean=0,
    init_std_dev=.1,
    biased=False,
    lr=.03,
    reg=.1,
    random_state=0,
    return_logs=False
)

df_train, df_test = svd(df_train, df_test, **hparams)
df_test['rating'] = df_test['rating'].apply(lambda x: (x>3)*(x-3))

/home/ilya/repos/recsys/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def get_item(i, dataset, user_based):
    if user_based:
        mask = dataset.user_id == i
    else:
        mask = dataset.item_id == i

    batch = dataset[mask]
    y_true = batch['rating'].to_numpy()
    y_score = batch['pred_rating'].to_numpy()

    return y_true, y_score

In [4]:
user_ids = df_test.user_id.unique()

targets = {}
preds = {}
for i in user_ids:
    y_true, y_score = get_item(i, df_test, user_based=True)
    if y_true.sum() == 0:
        continue
    targets[i] = y_true
    preds[i] = y_score

In [5]:
from ranking.metrics import MAP, MRR, NDCG

print('MAP@10:', MAP(targets, preds))
print('MRR:', MRR(targets, preds))
print('NDCG@10:', NDCG(targets, preds))

MAP@10: 0.7943695821079642
MRR: 0.8801073213011116
NDCG@10: 0.8268825480566907


In [6]:
print('MAP@5:', MAP(targets, preds, k=5))
print('NDCG@5:', NDCG(targets, preds, k=5))

MAP@5: 0.840051748750895
NDCG@5: 0.8447014644841818


In [7]:
print('MAP@3:', MAP(targets, preds, k=3))
print('NDCG@3:', NDCG(targets, preds, k=3))

MAP@3: 0.8570663811563188
NDCG@3: 0.8519743548704194
